In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("Brazilian Dataset").getOrCreate()

22/12/23 12:58:50 WARN Utils: Your hostname, Nishantkumars-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.29.169.121 instead (on interface en0)
22/12/23 12:58:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/23 12:58:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path = "../pyspark/archive/olist_customers_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_customers_dataset")

In [4]:
path = "../pyspark/archive/olist_geolocation_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_geolocation_dataset")

In [5]:
path = "../pyspark/archive/olist_order_items_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_order_items_dataset")

In [6]:
path = "../pyspark/archive/olist_order_payments_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_order_payments_dataset")

In [7]:
path = "../pyspark/archive/olist_order_reviews_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_order_reviews_dataset")

In [8]:
path = "../pyspark/archive/olist_orders_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_orders_dataset")

In [9]:
path = "../pyspark/archive/olist_products_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_products_dataset")

In [10]:
path = "../pyspark/archive/olist_sellers_dataset.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("olist_sellers_dataset")

In [11]:
path = "../pyspark/archive/product_category_name_translation.csv"
df= spark.read.option("header", True).csv(path)
df.createOrReplaceTempView("product_category_name_translation")

In [ ]:
## 1. Find top 10 cities with maximum number of customers?
sqlDF = spark.sql("SELECT customer_city,count(*) as total FROM olist_customers_dataset GROUP BY customer_city ORDER BY total desc LIMIT 10")
sqlDF.show()

In [ ]:
## 2. Find product (with count) with product category and product_id that is ordered maximum number of times in each month year (eg. 2020-01)? 
sqlDF = spark.sql("SELECT order_id,concat(MONTH(order_purchase_timestamp),' ',YEAR(order_purchase_timestamp)) as month_year FROM olist_orders_dataset")
sqlDF.createOrReplaceTempView("temp")
sqlDF = spark.sql("SELECT order_id,olist_order_items_dataset.product_id,product_category_name FROM olist_order_items_dataset INNER JOIN olist_products_dataset ")
sqlDF.createOrReplaceTempView("temp1")
sqlDF = spark.sql("SELECT product_id,product_category_name,month_year,count(*) as total FROM temp INNER JOIN temp1 GROUP BY product_id,product_category_name,month_year")
sqlDF.createOrReplaceTempView("temp")
sqldf=spark.sql("SELECT DISTINCT month_year,FIRST_VALUE(product_id) OVER (PARTITION BY month_year ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as prod_id,FIRST_VALUE(product_category_name) OVER (PARTITION BY month_year ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as product_category,FIRST_VALUE(total) OVER (PARTITION BY month_year ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as total from temp")
sqldf.show()

22/12/22 14:01:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 428975 ms exceeds timeout 120000 ms
22/12/22 14:01:23 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
## 3. Find product (with count) with product category and product_id that is ordered maximum/minimum number of times in each customer city? 
sqlDF = spark.sql("SELECT order_id,customer_city FROM olist_orders_dataset INNER JOIN olist_customers_dataset")
sqlDF.createOrReplaceTempView("temp")
sqlDF = spark.sql("SELECT olist_order_items_dataset.product_id,order_id,product_category_name FROM olist_order_items_dataset INNER JOIN olist_products_dataset")
sqlDF.createOrReplaceTempView("temp3")
sqlDF = spark.sql("SELECT customer_city,product_id,product_category_name,count(*) as total FROM temp INNER JOIN temp3 GROUP BY customer_city,product_id,product_category_name")
sqlDF.createOrReplaceTempView("temp")
sqldf=spark.sql("SELECT DISTINCT customer_city,FIRST_VALUE(product_id) OVER (PARTITION BY customer_city ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as min_prod_id,FIRST_VALUE(product_category_name) OVER (PARTITION BY customer_city ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as min_product_category,FIRST_VALUE(total) OVER (PARTITION BY customer_city ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as min_order from temp")
sqldf.createOrReplaceTempView("temp1")
sqldf=spark.sql("SELECT DISTINCT customer_city,LAST_VALUE(product_id) OVER (PARTITION BY customer_city ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as max_prod_id,LAST_VALUE(product_category_name) OVER (PARTITION BY customer_city ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as max_product_category,LAST_VALUE(total) OVER (PARTITION BY customer_city ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as max_order from temp")
sqldf.createOrReplaceTempView("temp2")
spark.sql("SELECT temp1.customer_city,min_prod_id,min_product_category,min_order,max_prod_id,max_product_category,max_order from temp1 INNER JOIN temp2 ").show()

In [ ]:
## 4. Find the payment type that is used the most/least in a particular year?
sqlDF = spark.sql("SELECT YEAR(order_purchase_timestamp) as year,payment_type,count(*) as total from olist_orders_dataset INNER JOIN olist_order_payments_dataset GROUP BY YEAR(order_purchase_timestamp),payment_type HAVING payment_type!='not_defined'")
sqlDF.createOrReplaceTempView("temp")
sqldf=spark.sql("SELECT DISTINCT year,FIRST_VALUE(payment_type) OVER (PARTITION BY year ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as MIN_PAYMENT_TYPE,FIRST_VALUE(total) OVER (PARTITION BY year ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as min_value from temp")
sqldf.createOrReplaceTempView("temp1") 
sqldF=spark.sql("SELECT DISTINCT year,LAST_VALUE(payment_type) OVER (PARTITION BY year ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) as MAX_PAYMENT_TYPE,LAST_VALUE(total) OVER (PARTITION BY year ORDER BY total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as max_value from temp")
sqldF.createOrReplaceTempView("temp2")
spark.sql("SELECT temp1.year,MIN_PAYMENT_TYPE,min_value,MAX_PAYMENT_TYPE,max_value from (temp1 INNER JOIN temp2 ON temp1.year=temp2.year)").show()

In [ ]:
## 5. Find top 5 sellers with highest rated products. Include average rating in the output given that seller must have sold atleast 10 products ?
sqlDF = spark.sql("SELECT product_id,seller_id, AVG(review_score) as score, count(*) as total FROM olist_order_items_dataset INNER JOIN olist_order_reviews_dataset GROUP BY seller_id,product_id HAVING total >=10 ORDER BY score DESC LIMIT 5")
sqlDF.show()

In [43]:
## 6. Percentage of customers opt for payment in installments in each year? 
sqlDF = spark.sql("SELECT YEAR(order_purchase_timestamp) as year,CASE WHEN payment_installments > 1 THEN 'A' ELSE 'B' END as RES FROM olist_order_payments_dataset INNER JOIN olist_orders_dataset ")
sqlDF.createOrReplaceTempView("temp")
sqlDF = spark.sql("SELECT year,RES,count(*) as total, SUM(count(*)) OVER(PARTITION BY year) gtotal FROM temp GROUP BY year,RES") 
sqlDF.createOrReplaceTempView("temp")
spark.sql(" SELECT year,total,gtotal,total/gtotal from temp WHERE RES='A' ").show()


+----+----------+----------+------------------+
|year|     total|    gtotal|  (total / gtotal)|
+----+----------+----------+------------------+
|2016|  16890202|  34178494|0.4941763086460158|
|2016|  17288292|  34178494|0.5058236913539842|
|2017|2369967348|4685362486|0.5058236913539842|
|2017|2315395138|4685362486|0.4941763086460158|
|2018|2772816718|5610986746|0.4941763086460158|
|2018|2838170028|5610986746|0.5058236913539842|
+----+----------+----------+------------------+



In [ ]:
## 7. Find top 5 months with maximum products sold across all years? 
sqlDF=spark.sql("SELECT YEAR(order_purchase_timestamp) as year,MONTH(order_purchase_timestamp) as month,count(*) as total from olist_orders_dataset GROUP BY YEAR(order_purchase_timestamp),MONTH(order_purchase_timestamp)")
sqlDF.createOrReplaceTempView("temp")
sqlDF=spark.sql("SELECT year,month,total,ROW_NUMBER() OVER(PARTITION BY year ORDER BY total DESC) as RNO from temp") 
sqlDF.createOrReplaceTempView("temp1")
spark.sql("SELECT * FROM temp1 WHERE RNO<=5").show()

In [ ]:
## 8. Find top 5 cities from where maximum products are sold? 
sqlDF=spark.sql("SELECT seller_city,count(*) as total FROM olist_order_items_dataset INNER JOIN olist_sellers_dataset GROUP BY seller_city ORDER BY total DESC")
sqlDF.show(5)

In [ ]:
## 9. Find top 10 products with maximum review score or ratings? 
sqlDF=spark.sql("SELECT order_id,product_category_name FROM olist_products_dataset INNER JOIN olist_order_items_dataset"); 
sqlDF.createOrReplaceTempView("temp")
sqlDF=spark.sql("SELECT product_category_name, AVG(review_score) as score FROM olist_order_reviews_dataset INNER JOIN temp GROUP BY product_category_name ORDER BY score DESC LIMIT 10")
sqlDF.createOrReplaceTempView("temp")
spark.sql("SELECT * FROM temp").show()

In [ ]:
## 10. Find the count of total payments made through each payment_type by Year? 
sqlDF = spark.sql("SELECT YEAR(order_purchase_timestamp) as year,payment_type,sum(payment_value*payment_installments) as total_payment from olist_orders_dataset INNER JOIN olist_order_payments_dataset GROUP BY year,payment_type")
sqlDF.show()

In [ ]:
## 11. Find top 10 customer_city with maximum orders delivered? 
sqlDF = spark.sql("SELECT customer_city, count(*) as total from olist_orders_dataset INNER JOIN  olist_customers_dataset ON olist_orders_dataset.customer_id=olist_customers_dataset.customer_id AND order_status='delivered' GROUP BY customer_city ORDER BY total DESC LIMIT 10")
sqlDF.show()

In [ ]:
## 12. Find top 10 sellers with maximum revenue (price) earned in a given year? 
sqlDF = spark.sql("SELECT YEAR(order_purchase_timestamp) as year,seller_id,SUM(price) as total from olist_orders_dataset INNER JOIN  olist_order_items_dataset ON olist_orders_dataset.order_id=olist_order_items_dataset.order_id AND order_status='delivered' GROUP BY YEAR(order_purchase_timestamp),seller_id")
sqlDF.createOrReplaceTempView("temp")
sqlDF = spark.sql("SELECT year,seller_id,total, ROW_NUMBER() OVER(PARTITION BY year ORDER BY total DESC) as RNO from temp")
sqlDF.createOrReplaceTempView("temp")
spark.sql("SELECT year,seller_id,total,RNO from temp WHERE RNO<=10").show() 

In [ ]:
## 13. Find top 10 cusomers for a given product_category?
sqlDF = spark.sql("SELECT olist_order_items_dataset.order_id,product_category_name FROM olist_products_dataset INNER JOIN olist_order_items_dataset")
sqlDF.createOrReplaceTempView("temp") 
sqlDF = spark.sql("SELECT customer_id,product_category_name,count(*) as total FROM temp INNER JOIN olist_orders_dataset GROUP BY customer_id,product_category_name")
sqlDF.createOrReplaceTempView("temp") 
sqlDF = spark.sql("SELECT customer_id,product_category_name,total,ROW_NUMBER() OVER (PARTITION BY product_category_name ORDER BY total DESC) as RNO FROM temp")
sqlDF.createOrReplaceTempView("temp1")
spark.sql("SELECT * FROM temp1 WHERE RNO<=10").show()

In [ ]:
## 14. Find top 10 sellers for a given product_category? 
sqlDF = spark.sql("SELECT olist_order_items_dataset.seller_id,product_category_name,count(*) as total FROM olist_products_dataset INNER JOIN olist_order_items_dataset GROUP BY seller_id,product_category_name")
sqlDF.createOrReplaceTempView("temp") 
sqlDF = spark.sql("SELECT seller_id,product_category_name,total,ROW_NUMBER() OVER(PARTITION BY product_category_name ORDER BY total DESC) as RNO FROM temp")
sqlDF.createOrReplaceTempView("temp") 
spark.sql("SELECT * FROM temp WHERE RNO<=10").show()